# LLM如何基于SQL做QA（二）

一篇已经讲了如何基于SQL做QA，但是那篇文章只是简单的介绍了一下，这篇会介绍通过一些方式来让LLM更好的理解SQL。
在开始之前，我们先准备下环境，和上一章一样，具体看上一章。

In [2]:
from langchain_community.utilities import SQLDatabase


db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

## 在Prompt中增加Example

LangChain提供了封装，可以支持sql的方言(dialects)，会自动做处理，这样就可以在prompt中增加example，让LLM更好的理解SQL。
`create_sql_query_chain`和`SQLDatabase`都会自动处理以下任何方言：

In [3]:
from langchain.chains.sql_database.prompt import SQL_PROMPTS

list(SQL_PROMPTS)

['crate',
 'duckdb',
 'googlesql',
 'mssql',
 'mysql',
 'mariadb',
 'oracle',
 'postgresql',
 'sqlite',
 'clickhouse',
 'prestodb']

当前使用的时候的是sqlite，所以我们可以使用sqlite的prompt。

In [4]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

我们需要将表的结构传递给模型模型，这样模型才能更好的生成SQL。LangChain提供了一些方法来拿到表结构，同时还可以生成数据示例。 

In [5]:
context = db.get_context()
print(list(context))
print("*"*10)
print(context["table_info"])
print("*"*10)
print(context["table_names"])

['table_info', 'table_names']
**********

CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOR

注意看，table_info中已经包含了表的结构，数据示例
完整的Prompt如下：

In [6]:
prompt_with_context = chain.get_prompts()[0].partial(table_info=context["table_info"])
print(prompt_with_context.pretty_repr())

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

这里有个问题，可以看到，这会将所有的表结构传递给模型，这样会导致模型的输入过大，可能会超过模型的上下文（当然在现在也不是问题，可以一块传递），还有一些优化方式来减少传递的数据，比如只传递部分表结构，或者只传递部分表的数据示例。下面会介绍如何做。  

### 增加示例

In [7]:
examples = [
    {"input": "List all artists.", "query": "SELECT * FROM Artist;"},
    {
        "input": "Find all albums for the artist 'AC/DC'.",
        "query": "SELECT * FROM Album WHERE ArtistId = (SELECT ArtistId FROM Artist WHERE Name = 'AC/DC');",
    },
    {
        "input": "List all tracks in the 'Rock' genre.",
        "query": "SELECT * FROM Track WHERE GenreId = (SELECT GenreId FROM Genre WHERE Name = 'Rock');",
    },
    {
        "input": "Find the total duration of all tracks.",
        "query": "SELECT SUM(Milliseconds) FROM Track;",
    },
    {
        "input": "List all customers from Canada.",
        "query": "SELECT * FROM Customer WHERE Country = 'Canada';",
    },
    {
        "input": "How many tracks are there in the album with ID 5?",
        "query": "SELECT COUNT(*) FROM Track WHERE AlbumId = 5;",
    },
    {
        "input": "Find the total number of invoices.",
        "query": "SELECT COUNT(*) FROM Invoice;",
    },
    {
        "input": "List all tracks that are longer than 5 minutes.",
        "query": "SELECT * FROM Track WHERE Milliseconds > 300000;",
    },
    {
        "input": "Who are the top 5 customers by total purchase?",
        "query": "SELECT CustomerId, SUM(Total) AS TotalPurchase FROM Invoice GROUP BY CustomerId ORDER BY TotalPurchase DESC LIMIT 5;",
    },
    {
        "input": "Which albums are from the year 2000?",
        "query": "SELECT * FROM Album WHERE strftime('%Y', ReleaseDate) = '2000';",
    },
    {
        "input": "How many employees are there",
        "query": 'SELECT COUNT(*) FROM "Employee"',
    },
]

In [8]:
# 创建 FewShotPrompt
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate.from_template("User input: {input}\nSQL query: {query}")
prompt = FewShotPromptTemplate(
    examples=examples[:5],
    example_prompt=example_prompt,
    prefix="You are a SQLite expert. Given an input question, create a syntactically correct SQLite query to run. Unless otherwise specificed, do not return more than {top_k} rows.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries.",
    suffix="User input: {input}\nSQL query: ",
    input_variables=["input", "top_k", "table_info"],
)

In [9]:
print(prompt.format(input="How many artists are there?", top_k=3, table_info="foo"))

You are a SQLite expert. Given an input question, create a syntactically correct SQLite query to run. Unless otherwise specificed, do not return more than 3 rows.

Here is the relevant table info: foo

Below are a number of examples of questions and their corresponding SQL queries.

User input: List all artists.
SQL query: SELECT * FROM Artist;

User input: Find all albums for the artist 'AC/DC'.
SQL query: SELECT * FROM Album WHERE ArtistId = (SELECT ArtistId FROM Artist WHERE Name = 'AC/DC');

User input: List all tracks in the 'Rock' genre.
SQL query: SELECT * FROM Track WHERE GenreId = (SELECT GenreId FROM Genre WHERE Name = 'Rock');

User input: Find the total duration of all tracks.
SQL query: SELECT SUM(Milliseconds) FROM Track;

User input: List all customers from Canada.
SQL query: SELECT * FROM Customer WHERE Country = 'Canada';

User input: How many artists are there?
SQL query: 


### 动态的few-shot prompt
如果我们有足够的例子，我们可能只想在提示中包含最相关的例子，要么是因为它们不适合模型的上下文窗口，要么是因为长尾的例子会分散模型的注意力。所以，对于任何输入，我们希望包含与该输入最相关的示例。

可以使用`SemanticSimilarityExampleSelector`,它会根据输入的相似度来选择最相关的例子。底层是将数据放在向量数据库中，然后使用向量相似度来选择最相关的例子。


In [10]:
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    Chroma,
    k=5,
    input_keys=["input"],
)

In [11]:
example_selector.select_examples({"input": "how many artists are there?"})

[{'input': 'List all artists.', 'query': 'SELECT * FROM Artist;'},
 {'input': 'How many employees are there',
  'query': 'SELECT COUNT(*) FROM "Employee"'},
 {'input': 'How many tracks are there in the album with ID 5?',
  'query': 'SELECT COUNT(*) FROM Track WHERE AlbumId = 5;'},
 {'input': 'Which albums are from the year 2000?',
  'query': "SELECT * FROM Album WHERE strftime('%Y', ReleaseDate) = '2000';"},
 {'input': "List all tracks in the 'Rock' genre.",
  'query': "SELECT * FROM Track WHERE GenreId = (SELECT GenreId FROM Genre WHERE Name = 'Rock');"}]

可以看到已经选择好了,可以代入到`FewShotPromptTemplate`中如下

In [12]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="You are a SQLite expert. Given an input question, create a syntactically correct SQLite query to run. Unless otherwise specificed, do not return more than {top_k} rows.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries.",
    suffix="User input: {input}\nSQL query: ",
    input_variables=["input", "top_k", "table_info"],
)

In [13]:
print(prompt.format(input="how many artists are there?", top_k=3, table_info="foo"))

You are a SQLite expert. Given an input question, create a syntactically correct SQLite query to run. Unless otherwise specificed, do not return more than 3 rows.

Here is the relevant table info: foo

Below are a number of examples of questions and their corresponding SQL queries.

User input: List all artists.
SQL query: SELECT * FROM Artist;

User input: How many employees are there
SQL query: SELECT COUNT(*) FROM "Employee"

User input: How many tracks are there in the album with ID 5?
SQL query: SELECT COUNT(*) FROM Track WHERE AlbumId = 5;

User input: Which albums are from the year 2000?
SQL query: SELECT * FROM Album WHERE strftime('%Y', ReleaseDate) = '2000';

User input: List all tracks in the 'Rock' genre.
SQL query: SELECT * FROM Track WHERE GenreId = (SELECT GenreId FROM Genre WHERE Name = 'Rock');

User input: how many artists are there?
SQL query: 


## 验证查询和处理无效查询

### 验证查询
原理：增加一个新的chain，用来验证查询，如果查询有问题，就重写查询，如果没有问题，就直接返回查询。

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

system = """Double check the user's {dialect} query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

If there are any of the above mistakes, rewrite the query.
If there are no mistakes, just reproduce the original query with no further commentary.

Output the final SQL query only."""
prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{query}")]
).partial(dialect=db.dialect)
validation_chain = prompt | llm | StrOutputParser()

full_chain = {"query": chain} | validation_chain

In [15]:
from langchain.globals import set_debug
set_debug(True)
query = full_chain.invoke(
    {
        "question": "What's the average Invoice from an American customer whose Fax is missing since 2003 but before 2010"
    }
)
print(query)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "question": "What's the average Invoice from an American customer whose Fax is missing since 2003 but before 2010"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<query>] Entering Chain run with input:
{
  "question": "What's the average Invoice from an American customer whose Fax is missing since 2003 but before 2010"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<query> > chain:RunnableSequence] Entering Chain run with input:
{
  "question": "What's the average Invoice from an American customer whose Fax is missing since 2003 but before 2010"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<query> > chain:RunnableSequence > chain:RunnableAssign<input,table_info>] Entering Chain run with input:
{
  "question": "What's the average Invoice from an American customer whose Fax is missing since 2003 but before 2010"
}
[chain/start] [chain:RunnableSequence > chain:Run

langsmith地址：https://smith.langchain.com/public/f761d577-b89b-4944-9faa-c07c0df89c6b/r

这个方法的缺点是，需要进行两次模型调用，可以在一次模型调用中完成，下面会介绍如何做。
其实就是将两个chain中的promot合并为一个Prompt

In [16]:
system = """You are a {dialect} expert. Given an input question, create a syntactically correct {dialect} query to run.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per {dialect}. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Only use the following tables:
{table_info}

Write an initial draft of the query. Then double check the {dialect} query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

Use format:

First draft: <<FIRST_DRAFT_QUERY>>
Final answer: <<FINAL_ANSWER_QUERY>>
"""
prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{input}")]
).partial(dialect=db.dialect)


def parse_final_answer(output: str) -> str:
    return output.split("Final answer: ")[1]


chain = create_sql_query_chain(llm, db, prompt=prompt) | parse_final_answer
prompt.pretty_print()

================================ System Message ================================

You are a {dialect} expert. Given an input question, create a syntactically correct {dialect} query to run.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per {dialect}. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Only use the following tables:
{table_info}

Write an initial draft of the query. Then 

In [17]:
query = chain.invoke(
    {
        "question": "What's the average Invoice from an American customer whose Fax is missing since 2003 but before 2010"
    }
)
print(query)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "question": "What's the average Invoice from an American customer whose Fax is missing since 2003 but before 2010"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<input,table_info>] Entering Chain run with input:
{
  "question": "What's the average Invoice from an American customer whose Fax is missing since 2003 but before 2010"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<input,table_info> > chain:RunnableParallel<input,table_info>] Entering Chain run with input:
{
  "question": "What's the average Invoice from an American customer whose Fax is missing since 2003 but before 2010"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<input,table_info> > chain:RunnableParallel<input,table_info> > chain:RunnableLambda] Entering Chain run with input:
{
  "question": "What's the average Invoice from an American customer whose Fax is missing since 2003 but before 2010"
}
[cha

langsmith地址：https://smith.langchain.com/public/339a1303-f015-4b19-bd95-916dc35e5c5e/r

### 处理无效查询

建议还是人为的处理，因为无效查询的原因太多了，模型无法处理所有的情况。并且不能保证生产的sql是绝对安全的，需要人为介入。

## 处理大量表的情况

在前面讲过，产生这个问题的背景，下面会介绍如何处理这个问题。
其实这里的思想还是计算机中经典的思想，分治法，将大问题分解为小问题，然后递归的解决小问题，最后合并结果。


### 将问题归类到相关的表中（一次性拿到所有的表）

有个点。
我们需要在Prompt中包含的主要信息是相关表的schema。当我们有很多表时，我们无法将所有schema都放在一个Prompt中。在这种情况下，我们可以首先提取与用户输入相关的表的名称，然后只包含它们的schema。


In [18]:
## 第一步，通过问题找到相关的表
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field


class Table(BaseModel):
    """Table in SQL database."""

    name: str = Field(description="Name of table in SQL database.")


table_names = "\n".join(db.get_usable_table_names())
system = f"""Return the names of ALL the SQL tables that MIGHT be relevant to the user question. \
The tables are:

{table_names}

Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{input}"),
    ]
)
llm_with_tools = llm.bind_tools([Table])
output_parser = PydanticToolsParser(tools=[Table])

table_chain = prompt | llm_with_tools | output_parser

table_chain.invoke({"input": "What are all the genres of Alanis Morisette songs"})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "What are all the genres of Alanis Morisette songs"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "input": "What are all the genres of Alanis Morisette songs"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Return the names of ALL the SQL tables that MIGHT be relevant to the user question. The tables are:\n\nAlbum\nArtist\nCustomer\nEmployee\nGenre\nInvoice\nInvoiceLine\nMediaType\nPlaylist\nPlaylistTrack\nTrack\n\nRemember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed.\nHuman: What are all the genres of Alanis Morisette songs"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generation

[Table(name='Genre')]

我们已经找到了相关的表，还有一种方式，可以对表做分组，然后根据分组来找到相关的表。如下：
### 将问题归类到相关的表中（做归类查询，先将表做归类）

In [19]:
system = """Return the names of any SQL tables that are relevant to the user question.
The tables are:

Music
Business
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{input}"),
    ]
)

category_chain = prompt | llm_with_tools | output_parser
category_chain.invoke({"input": "What are all the genres of Alanis Morisette songs"})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "What are all the genres of Alanis Morisette songs"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "input": "What are all the genres of Alanis Morisette songs"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Return the names of any SQL tables that are relevant to the user question.\nThe tables are:\n\nMusic\nBusiness\n\nHuman: What are all the genres of Alanis Morisette songs"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] [1.44s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_info": {
          "finish_reason": "tool_calls",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "m

[Table(name='Music'), Table(name='Business')]

In [20]:
# 上一步中拿到了分类，其实就是对表做了归类，下面在继续找，从分类中找

from typing import List

## 从分类中找
def get_tables(categories: List[Table]) -> List[str]:
    tables = []
    for category in categories:
        if category.name == "Music":
            tables.extend(
                [
                    "Album",
                    "Artist",
                    "Genre",
                    "MediaType",
                    "Playlist",
                    "PlaylistTrack",
                    "Track",
                ]
            )
        elif category.name == "Business":
            tables.extend(["Customer", "Employee", "Invoice", "InvoiceLine"])
    return tables

## 链接到chain中。
table_chain = category_chain | get_tables
table_chain.invoke({"input": "What are all the genres of Alanis Morisette songs"})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "What are all the genres of Alanis Morisette songs"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "input": "What are all the genres of Alanis Morisette songs"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Return the names of any SQL tables that are relevant to the user question.\nThe tables are:\n\nMusic\nBusiness\n\nHuman: What are all the genres of Alanis Morisette songs"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] [1.24s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_info": {
          "finish_reason": "tool_calls",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "m

['Album',
 'Artist',
 'Genre',
 'MediaType',
 'Playlist',
 'PlaylistTrack',
 'Track',
 'Customer',
 'Employee',
 'Invoice',
 'InvoiceLine']

这里已经找到了相关的表，我们再来回顾下，先找到分类，在通过分类找到表，这样就可以减少传递的数据，减少模型的输入。

In [21]:
from operator import itemgetter

from langchain.chains import create_sql_query_chain
from langchain_core.runnables import RunnablePassthrough

query_chain = create_sql_query_chain(llm, db)
# Convert "question" key to the "input" key expected by current table_chain.
table_chain = {"input": itemgetter("question")} | table_chain
# Set table_names_to_use using table_chain.
full_chain = RunnablePassthrough.assign(table_names_to_use=table_chain) | query_chain

In [22]:
query = full_chain.invoke(
    {"question": "What are all the genres of Alanis Morisette songs"}
)
print(query)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "question": "What are all the genres of Alanis Morisette songs"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<table_names_to_use>] Entering Chain run with input:
{
  "question": "What are all the genres of Alanis Morisette songs"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<table_names_to_use> > chain:RunnableParallel<table_names_to_use>] Entering Chain run with input:
{
  "question": "What are all the genres of Alanis Morisette songs"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<table_names_to_use> > chain:RunnableParallel<table_names_to_use> > chain:RunnableSequence] Entering Chain run with input:
{
  "question": "What are all the genres of Alanis Morisette songs"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<table_names_to_use> > chain:RunnableParallel<table_names_to_use> > chain:RunnableSequence > chain:RunnableParallel<input>] Entering Cha

langsmith：https://smith.langchain.com/public/05f006ca-9d84-43e5-baa2-79a948971f24/r
这里是将上面两个chain连接起来了。

In [23]:
# 执行查询
db.run(query)

"[('Rock',)]"

## 处理表中高基数列

高基数列其实就是包含了很多专有名词的列，在上一篇文章中说过他的处理方式

比如作者的名字，用户的问题可能是 张国荣的歌有什么？比如我们有一个auths的表，对应的sql应该是 `select  * from auths where name='张国荣'`
上面说的是理想的情况，实际中用户的输入的问题可能是  张国立的歌有什么？ 我们期望生成的sql也应该是上面那样。这种专有名词的列，叫做高基数列。
处理方式：
 将专有名字做向量化，在传递给模型之前， 先做一遍向量化，拿到相似的top5的名字，传递给模型来做。

In [16]:
# 构建专有名词
import ast
import re


def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return res


proper_nouns = query_as_list(db, "SELECT Name FROM Artist")
proper_nouns += query_as_list(db, "SELECT Title FROM Album")
proper_nouns += query_as_list(db, "SELECT Name FROM Genre")
len(proper_nouns)
proper_nouns[:5]

['AC/DC', 'Accept', 'Aerosmith', 'Alanis Morissette', 'Alice In Chains']

将上面的词建立向量化数据库

In [19]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

## 建立向量化数据库
vector_db = Chroma.from_texts(proper_nouns, OpenAIEmbeddings())
retriever = vector_db.as_retriever(search_kwargs={"k": 15})

将retrieve和chain结合起来

In [26]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

system = """You are a SQLite expert. Given an input question, create a syntactically
correct SQLite query to run. Unless otherwise specificed, do not return more than
{top_k} rows.

Only return the SQL query with no markup or explanation.

Here is the relevant table info: {table_info}

Here is a non-exhaustive list of possible feature values. If filtering on a feature
value make sure to check its spelling against this list first:

{proper_nouns}
"""
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")])

query_chain = create_sql_query_chain(llm, db, prompt=prompt)
retriever_chain = (
    itemgetter("question")
    | retriever
    | (lambda docs: "\n".join(doc.page_content for doc in docs))
)
chain = RunnablePassthrough.assign(proper_nouns=retriever_chain) | query_chain

下面的是传入了一个错误的人名，最终没有查出来

In [27]:
query = query_chain.invoke(
    {"question": "What are all the genres of elenis moriset songs", "proper_nouns": ""}
)
print(query)
db.run(query)

SELECT DISTINCT g.Name
FROM Genre g
JOIN Track t ON g.GenreId = t.GenreId
JOIN Album a ON t.AlbumId = a.AlbumId
JOIN Artist ar ON a.ArtistId = ar.ArtistId
WHERE ar.Name = 'Elenis Moriset';


''

下面是结合了retriever_chain的样子，会对专业名词做了一个相似性搜索，会修正错误的单词

In [35]:
# With retrieval
query = chain.invoke({"question": "What are all the genres of eleni moriset songs"})
print(query)
db.run(query)

SELECT DISTINCT g.Name
FROM Genre g
JOIN Track t ON g.GenreId = t.GenreId
JOIN Album a ON t.AlbumId = a.AlbumId
WHERE a.Title LIKE '%Eleni Moriset%';


''

langsmith：https://smith.langchain.com/public/dfac0151-eea6-4a30-8881-f3c0de443772/r
注意看这里的执行过程，这里去做了retrieve

## 基于csv做问答
官方文档： https://python.langchain.com/v0.2/docs/how_to/sql_csv/
推荐的方式是将csv存储在sql中，变为结构化数据方便些，它比使用Python更容易限制权限。
LangChain有处理方式，是让模型返回python代码，我们提供一个python的运行环境，来运行python代码，这种方式在生产里面很不推荐，这里就不过多的介绍了。



**从后面开始，LangChain相关的技术了，开始LangGraph的学习**